In [72]:
import pandas as pd
import numpy as np
import rie_estimator
import csestimator
from hcaa_implementation import hcaa_alocation


In [5]:
precios_americano = pd.read_csv(
    "./sp_500_original_clean.csv",
    index_col="Date",
    parse_dates=True,
)

precios_europeo = pd.read_csv(
    "./european_market_original_clean.csv",
    index_col="Date",
    parse_dates=True,
)

retornos_american = (np.log(precios_americano) - np.log(precios_americano.shift(1))).iloc[1:]
retornos_europeo = (np.log(precios_europeo) - np.log(precios_europeo.shift(1))).iloc[1:]

In [10]:
retornos_american.shape

(2521, 460)

In [11]:
2521 - 920

1601

In [8]:
retornos_europeo.head()

,AD.AS,AGN.AS,AKZA.AS,ASM.AS,ASML.AS,BESI.AS,DSM.AS,HEIA.AS,INGA.AS,KPN.AS,...,NOVN.SW,PGHN.SW,ROG.SW,SCMN.SW,SGSN.SW,SIKA.SW,SLHN.SW,SREN.SW,UBSG.SW,ZURN.SW
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-04,-0.014088,-0.022437,-0.010624,-0.027017,-0.020262,0.002881,0.004196,-0.009320,-0.014286,-0.021930,...,0.001837,0.000000,0.004929,-0.003060,-0.009491,-0.007121,-0.002822,0.000407,0.003463,-0.001854
2012-01-05,-0.001469,-0.019362,-0.000924,0.015559,-0.005955,-0.003073,-0.004875,-0.000407,-0.035543,-0.021271,...,-0.001837,-0.030962,0.000614,-0.016292,-0.005738,-0.016071,-0.031585,-0.008172,-0.033395,-0.006982
2012-01-06,0.002447,0.001628,0.004214,0.019167,0.009387,0.000192,-0.005717,-0.008590,-0.008215,0.000230,...,0.005500,0.000000,0.009172,-0.000283,0.008278,-0.003358,-0.004678,-0.014463,-0.008977,-0.001870
2012-01-09,0.002928,-0.008166,0.001970,0.007650,0.008991,-0.009662,-0.008087,0.010217,0.006396,-0.001150,...,-0.011973,-0.009479,-0.004270,-0.004543,0.006951,0.008929,-0.026130,-0.005007,-0.035800,-0.007044
2012-01-10,0.009217,0.071199,0.023335,0.036579,0.002466,0.003876,0.022051,0.001084,0.052686,-0.006349,...,0.001867,0.003170,-0.002448,0.014690,0.019951,0.041888,0.009581,0.031298,0.034898,0.024210


In [22]:
retornos_american.shape

(2521, 460)

In [67]:
r_cuad_sample = []
r_cuad_rie = []
r_cuad_eca_2 = []
r_cuad_eca_5 = []
r_cuad_eca_8 = []
ipr_mark_sample = []
ipr_mark_rie = []
ipr_mark_eca_2 = []
ipr_mark_eca_5 = []
ipr_mark_eca_8 = []


# 2521
for i in range(920, 930):
    # elegir los retornos de la ventana de tiempo
    selected_returns = retornos_american.iloc[i- 920 :i, :]

    # obtener los distintos estimadores
    sample_estimator = np.corrcoef(selected_returns.T)
    rie_matrix = rie_estimator.get_rie(returns = selected_returns, normalize = False, max_ones=True)
    eca_alfa_2 = csestimator.get_shrinkage_est(X_matrix = selected_returns, alpha = 0.2)
    eca_alfa_5 = csestimator.get_shrinkage_est(X_matrix = selected_returns, alpha = 0.5)
    eca_alfa_8 = csestimator.get_shrinkage_est(X_matrix = selected_returns, alpha = 0.8)
    print(eca_alfa_2.shape)
    
    # se obtienen pesos de markowitz
    inverse_sample = np.linalg.inv(sample_estimator)
    inverse_rie = np.linalg.inv(rie_matrix)
    inverse_eca_2 = np.linalg.inv(eca_alfa_2)
    inverse_eca_5 = np.linalg.inv(eca_alfa_5)
    inverse_eca_8 = np.linalg.inv(eca_alfa_8)
    mark_weights_sample = (inverse_sample @ np.ones(460) )/(np.ones((1,460)) @ inverse_sample @ np.ones(460) )
    mark_weigth_rie = (inverse_rie @ np.ones(460) )/(np.ones((1,460)) @ inverse_rie @ np.ones(460) )
    mark_weigth_eca_2 = (inverse_eca_2 @ np.ones(460) )/(np.ones((1,460)) @ inverse_eca_2 @ np.ones(460) )
    mark_weigth_eca_5 = (inverse_eca_5 @ np.ones(460) )/(np.ones((1,460)) @ inverse_eca_5 @ np.ones(460) )
    mark_weigth_eca_8 = (inverse_eca_8 @ np.ones(460) )/(np.ones((1,460)) @ inverse_eca_8 @ np.ones(460) )

    

    r_cuad_sample.append( 1 / (np.ones((1,460)) @ inverse_sample @ np.ones(460)))
    r_cuad_rie.append(1 / (np.ones((1,460)) @ inverse_rie @ np.ones(460)))
    r_cuad_eca_2.append( 1 / (np.ones((1,460)) @ inverse_eca_2 @ np.ones(460)))
    r_cuad_eca_5.append( 1 / (np.ones((1,460)) @ inverse_eca_5 @ np.ones(460)))
    r_cuad_eca_8.append( 1 / (np.ones((1,460)) @ inverse_eca_8 @ np.ones(460)))
    # se calcula el IPR
    ipr_mark_sample.append((np.abs(mark_weights_sample)**4).sum())
    ipr_mark_rie.append((np.abs(mark_weigth_rie)**4).sum())
    ipr_mark_eca_2.append((np.abs(mark_weigth_eca_2)**4).sum())
    ipr_mark_eca_5.append((np.abs(mark_weigth_eca_5)**4).sum())
    ipr_mark_eca_8.append((np.abs(mark_weigth_eca_8)**4).sum())
    

    #pesos_mark = 

    #

(460, 460)
(460, 460)
(460, 460)
(460, 460)
(460, 460)
(460, 460)
(460, 460)
(460, 460)
(460, 460)
(460, 460)


In [71]:
ipr_mark_rie

[6.713153341447628e-05,
 6.859613303619632e-05,
 7.137107269782236e-05,
 7.125132218454067e-05,
 7.280302291752136e-05,
 7.225862068553382e-05,
 7.346686122104612e-05,
 7.195587721274728e-05,
 7.156772895945796e-05,
 7.081218332877443e-05]

In [69]:
ipr_mark_sample

[0.0025879749806747947,
 0.0025663985973995006,
 0.0023829352344040614,
 0.0016849250953365276,
 0.0016114165236562397,
 0.0012294753675322545,
 0.0014568565049703011,
 0.0009994786929651664,
 0.000862015237232485,
 0.0008732894051241926]

In [58]:
r_cuad_sample

[array([0.03585049]),
 array([0.03578093]),
 array([0.03582521]),
 array([0.03581353]),
 array([0.03582637]),
 array([0.03585791]),
 array([0.03581295]),
 array([0.03600424]),
 array([0.0360504]),
 array([0.03619362])]

In [59]:
r_cuad_eca_2

[array([0.08114249]),
 array([0.07993404]),
 array([0.07985565]),
 array([0.08215633]),
 array([0.080525]),
 array([0.08095027]),
 array([0.08216482]),
 array([0.0814412]),
 array([0.08021449]),
 array([0.08031024])]

In [60]:
r_cuad_eca_5

[array([0.11359143]),
 array([0.11056825]),
 array([0.11177122]),
 array([0.11428416]),
 array([0.11357193]),
 array([0.11405534]),
 array([0.11469005]),
 array([0.11433666]),
 array([0.1118081]),
 array([0.11201145])]

In [61]:
r_cuad_eca_8

[array([0.12671828]),
 array([0.12237987]),
 array([0.12473815]),
 array([0.12737308]),
 array([0.12677546]),
 array([0.12740488]),
 array([0.1279785]),
 array([0.12797706]),
 array([0.12536748]),
 array([0.12571895])]